In [11]:
import asyncio
import time
import threading
import socket
from contextlib import asynccontextmanager
from typing import Optional, Tuple
from kubernetes import client, config
from kubernetes.stream import portforward
import requests
from requests.exceptions import HTTPError
import docker.errors
from app.docker_client.clientContext import client as docker_client

In [70]:
def find_free_port():
    """Find a free port on the local machine"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

class AsyncPortForwardManager:
    """Manages Kubernetes port forwarding with explicit cleanup tracking"""
    
    def __init__(self, namespace: str, service_name: str, service_port: int, local_port: Optional[int] = None):
        self.namespace = namespace
        self.service_name = service_name
        self.service_port = service_port
        self.local_port = local_port or find_free_port()
        self.pf = None
        self._stop_event = threading.Event()
        self._pf_thread = None
        self._is_running = False
        self._cleanup_completed = False
        
    async def start(self) -> str:
        """Start port forwarding and return the local registry URL"""
        try:
            print(f"Starting port forward to {self.namespace}/{self.service_name}:{self.service_port} -> localhost:{self.local_port}")
            
            # Load kubernetes config
            config.load_config()
            
            # Initialize the API client
            api_client = client.ApiClient()
            core_v1 = client.CoreV1Api(api_client)
            
            # Get the service to verify it exists
            try:
                service = core_v1.read_namespaced_service(
                    name=self.service_name,
                    namespace=self.namespace
                )
                # Get selector (e.g., {"app": "my-app"})
                selector = service.spec.selector
                if not selector:
                    raise ValueError("Service does not have a selector")

                # Turn selector into label string: e.g., "app=my-app"
                label_selector = ",".join(f"{k}={v}" for k, v in selector.items())
                # List Pods matching selector
                pods = core_v1.list_namespaced_pod(namespace=self.namespace, label_selector=label_selector)
                if not pods.items:
                    raise RuntimeError("No pods found backing the service")
                pod_name = pods.items[0].metadata.name

                print(f"✓ Found service {self.service_name} in namespace {self.namespace}")
            except client.ApiException as e:
                if e.status == 404:
                    raise Exception(f"Service {self.service_name} not found in namespace {self.namespace}")
                raise
            
            # Create port forward
            self.pf = portforward(
                core_v1.connect_get_namespaced_pod_portforward,
                name=pod_name,
                namespace=self.namespace,
                ports=f"{self.local_port},{self.service_port}",
            )

            print(self.pf.websocket.connected)

            # Start port forward in a separate thread
            self._pf_thread = threading.Thread(target=self._run_port_forward, daemon=True)
            self._pf_thread.start()
            self._is_running = True
            
            # Wait for port forward to be ready
            await self._wait_for_port_forward()
            
            registry_url = f"localhost:{self.local_port}"
            print(f"✓ Port forward established: {registry_url} -> {self.service_name}:{self.service_port}")
            return registry_url
            
        except Exception as e:
            await self.stop()
            raise Exception(f"Failed to start port forward: {str(e)}")
    
    def _run_port_forward(self):
        """Run the port forward in a thread (this runs in sync context)"""
        try:
            while not self._stop_event.is_set():
                time.sleep(0.1)  # Keep this as sync since it's in a thread
        except Exception as e:
            print(f"Port forward thread error: {e}")
        finally:
            print("Port forward background thread stopped")
    
    async def _wait_for_port_forward(self, timeout: int = 15):
        """Wait for the port forward to be ready using async sleep"""
        print(f"Waiting for port forward to be ready on localhost:{self.local_port}...")
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            try:
                # Test connection to the local port
                print(f"Testing connection to localhost:{self.local_port} and {self.pf.websocket}...")
                with self.pf.websocket as s:
                    s.settimeout(1)
                    result = s.connect_ex(('localhost', self.local_port)) 
                    if result == 0:
                        # Port is open, now test the registry endpoint
                        test_url = f"http://localhost:{self.local_port}/v2/_catalog"
                        response = requests.get(test_url, timeout=2)
                        if response.status_code == 200:
                            print(f"✓ Registry is responding on localhost:{self.local_port}")
                            return
            except Exception as e:
                pass
            await asyncio.sleep(0.5)  # Use asyncio.sleep instead of time.sleep
        
        raise Exception(f"Port forward not ready after {timeout} seconds")
    
    async def stop(self):
        """Stop the port forward with explicit cleanup confirmation"""
        if self._cleanup_completed:
            print("Port forward already cleaned up")
            return
            
        print(f"🧹 Starting port forward cleanup for localhost:{self.local_port}")
        
        # Set stop event
        self._stop_event.set()
        self._is_running = False
        
        # Close port forward connection
        if self.pf:
            try:
                print("Closing port forward connection...")
                self.pf.close()
                print("✓ Port forward connection closed")
            except Exception as e:
                print(f"Error closing port forward: {e}")
        
        # Wait for thread to finish
        if self._pf_thread and self._pf_thread.is_alive():
            print("Waiting for port forward thread to stop...")
            try:
                # Wait for the thread to finish, but do it in an async way
                loop = asyncio.get_event_loop()
                await loop.run_in_executor(None, self._pf_thread.join, 10)
                print("✓ Port forward thread stopped")
            except Exception as e:
                print(f"Error stopping port forward thread: {e}")
        
        # Verify port is released
        await self._verify_port_released()
        
        self._cleanup_completed = True
        print(f"✅ Port forward cleanup completed for localhost:{self.local_port}")
    
    async def _verify_port_released(self, timeout: int = 5):
        """Verify that the local port has been released"""
        print(f"Verifying port {self.local_port} is released...")
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            try:
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                    s.settimeout(1)
                    result = s.connect_ex(('localhost', self.local_port))
                    if result != 0:  # Connection failed, port is free
                        print(f"✓ Port {self.local_port} is now free")
                        return
            except:
                print(f"✓ Port {self.local_port} is now free")
                return
            await asyncio.sleep(0.5)
        
        print(f"⚠️  Port {self.local_port} may still be in use")
    
    def is_running(self) -> bool:
        """Check if port forward is currently running"""
        return self._is_running and not self._cleanup_completed

In [4]:
from pydantic import BaseModel
from typing import Optional

# Add this to your existing models
class PushConfig(BaseModel):
    repository: str    # e.g., "myapp" 
    tag: str = "latest"
    namespace: str = "image-registry"  # Registry namespace
    service_name: str = "docker"       # Registry service name
    service_port: int = 5000           # Registry service port
    username: Optional[str] = None
    password: Optional[str] = None

In [24]:
async def push_image_by_id(image_id: str, push_config: PushConfig):
    """
    Tag an image by ID and push it to the Kubernetes-hosted registry with explicit cleanup
    """
    manager = None
    registry_url = None
    
    try:
        # Get the image by ID
        image = docker_client.images.get(image_id)
        print(f"📦 Found image {image_id} with tags: {image.tags}")
        
        # Manual port forward management for better control
        manager = AsyncPortForwardManager(
            namespace=push_config.namespace,
            service_name=push_config.service_name,
            service_port=push_config.service_port,
        )
        
        # Start port forwarding
        registry_url = await manager.start()
        
        # Create the full image name for the target registry
        full_image_name = f"{registry_url}/{push_config.repository}:{push_config.tag}"
        
        print(f"🏷️  Tagging image {image_id} as {full_image_name}")
        
        # Tag the image with the new name
        image.tag(f"{registry_url}/{push_config.repository}", tag=push_config.tag)
        
        # Prepare auth config if credentials provided
        auth_config = None
        if push_config.username and push_config.password:
            auth_config = {
                'username': push_config.username,
                'password': push_config.password
            }
            print("🔐 Using authentication credentials")
        
        print(f"🚀 Pushing image to {full_image_name}")
        
        # Push the image (run in executor to not block)
        loop = asyncio.get_event_loop()
        push_result = await loop.run_in_executor(
            None,
            lambda: docker_client.images.push(
                f"{registry_url}/{push_config.repository}",
                tag=push_config.tag,
                auth_config=auth_config,
                stream=True
            )
        )
        
        # Collect push logs
        logs = []
        for log_line in push_result:
            logs.append(log_line)
            print(f"Push log: {log_line}")
        
        print(f"✅ Successfully pushed image to {full_image_name}")
        
        return {
            "success": True,
            "image_id": image_id,
            "pushed_as": full_image_name,
            "registry_url": registry_url,
            "logs": logs
        }
        
    except docker.errors.ImageNotFound:
        raise Exception(f"Image with ID {image_id} not found")
    except docker.errors.APIError as e:
        raise Exception(f"Docker API error during push: {str(e)}")
    except Exception as e:
        raise Exception(f"Failed to push image: {str(e)}")
    
    finally:
        # CRITICAL: Always cleanup port forwarding
        if manager:
            print("🧹 Ensuring port forward cleanup...")
            try:
                await manager.stop()
                print("✅ Port forward cleanup completed")
            except Exception as cleanup_error:
                print(f"⚠️  Error during port forward cleanup: {cleanup_error}")
                # Don't raise here, just log the error

In [71]:
import nest_asyncio
import asyncio

# Allow nested event loops
nest_asyncio.apply()

async def main():
    result = await push_image_by_id("sha256:9aa7e6755ed042a0184f0126492d7eef871123b5878b701e3331d34769728995", PushConfig(
        repository="github-webhook_test",
        tag="859d62db",
        namespace="image-registry",
        service_name="docker",
        service_port=5000,
    ))
    return result

result = asyncio.run(main())
print(result)

📦 Found image sha256:9aa7e6755ed042a0184f0126492d7eef871123b5878b701e3331d34769728995 with tags: ['github-webhook_test:859d62db']
Starting port forward to image-registry/docker:5000 -> localhost:54806
✓ Found service docker in namespace image-registry
True
Waiting for port forward to be ready on localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localhost:54806...
Testing connection to localh

Exception: Failed to push image: Failed to start port forward: Port forward not ready after 15 seconds

In [65]:
from kubernetes import client, config
from kubernetes.stream import portforward

config.load_kube_config()
v1 = client.CoreV1Api()

s = portforward(
    v1.connect_get_namespaced_pod_portforward,
    name="docker-79565f58c9-zphwq",
    namespace="image-registry",
    ports="5002",  # This will forward podPort 8080 -> localPort 8080
)

In [67]:
with s as forwarder:
    print(f"Port forwarding {5000} -> {5000} for pod {"docker-79565f58c9-zphwq"} in namespace {"image-registry"}")
    # At this point, you can access http://localhost:8080
    # Your application or test code would go here
    # e.g., Make an HTTP request to localhost:8080

    # Keep the forwarding alive for a duration or until a condition is met
    import time
    time.sleep(60) # Forward for 60 seconds

print("Port forwarding stopped.")

TypeError: 'PortForward' object does not support the context manager protocol